In [25]:
%matplotlib inline

datapath = "../../data/dl1/obj_neural_code/all_flat/"
repath   = "../../data/dl1/obj_neural_code/rep/nofinetune/"
N = 1440

In [28]:
from time import time
from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import scipy.spatial.distance
from keras.models import Model
import utils; reload(utils)
from utils import plots
import vgg16; reload(vgg16)
from vgg16 import Vgg16
vgg = Vgg16()

In [11]:
batch_size = 60
batches = vgg.get_batches(datapath, batch_size=batch_size, shuffle=False)
nbatches = int(N/batch_size)
print('N.of batches ' + str(nbatches))

Found 1440 images belonging to 1 classes.
N.of batches 24


In [5]:
# create models

# lambda_1
# maxpooling2d_1
# maxpooling2d_2
# maxpooling2d_3
# maxpooling2d_4
# maxpooling2d_5
# dense_1
# dense_2
# dense_3

idx = [0,5,10,17,24,31,33,35,37]
base = vgg.model
layers  = []
for i in range(len(idx)):
    print(base.layers[idx[i]].name)
    layers.append( base.layers[idx[i]] )
    
models = []
for layer in layers:
    models.append( Model(input=base.input,
                         output=base.get_layer(layer.name).output) )
print('Done.')

lambda_1
maxpooling2d_1
maxpooling2d_2
maxpooling2d_3
maxpooling2d_4
maxpooling2d_5
dense_1
dense_2
dense_3
Done.


In [30]:
D = []
D_sq = []
for layer,model in zip(layers,models):
    print('Processing model : ' + layer.name)
    R = []
    batches = vgg.get_batches(datapath, batch_size=batch_size, shuffle=False)
    ti = time()
    for n in range(nbatches):
        imgs,_ = batches.next()       
        R.append(model.predict(imgs))
    R = np.asarray(R, dtype=float) 
    R.shape = [N,R.shape[2],R.shape[3],R.shape[4]]
    np.save(repath +  layer.name, R)
    
    # compute distances
    R.shape = R.shape[0], -1
    d = scipy.spatial.distance.pdist(R, 'euclidean')
    d_sq = scipy.spatial.distance.squareform(d, force='no', checks=True)
    D.append(d)
    D_sq.append(d_sq)
    te = time() - ti
    print(te) 
    
np.save(repath + 'D', D)
np.save(repath + 'D_sq', D_sq)
print('Done.')

Processing model : lambda_1
Found 1440 images belonging to 1 classes.
42.4226510525
Processing model : maxpooling2d_1
Found 1440 images belonging to 1 classes.
160.491375208
Processing model : maxpooling2d_2
Found 1440 images belonging to 1 classes.
100.189476013
Processing model : maxpooling2d_3
Found 1440 images belonging to 1 classes.
73.8215699196
Processing model : maxpooling2d_4
Found 1440 images belonging to 1 classes.
63.8503069878
Processing model : maxpooling2d_5
Found 1440 images belonging to 1 classes.
53.2669448853
Processing model : dense_1
Found 1440 images belonging to 1 classes.
50.8644709587
Processing model : dense_2
Found 1440 images belonging to 1 classes.
50.9312207699
Processing model : dense_3
Found 1440 images belonging to 1 classes.
50.6345911026
Done.
